In [24]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
#env_vars = !cat /content/.env
env_vars = open("/content/.env","r").read().split('\n')
for var in env_vars[:-1]:
    key, value = var.split(' = ')
    os.environ[key] = value

In [79]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#experiments with dask
import dask.dataframe as dd
import dask_geopandas as dg

container = 'ana-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/ana_station_metadata.csv'
station_df = pd.read_csv(station_url, storage_options=storage_options)                
#df = dd.read_csv(station_url, storage_options=storage_options)
#ddf = dg.from_dask_dataframe(df).set_geometry(dg.points_from_xy(ddf, 'Latitude', 'Longitude'))

In [80]:
#copy from old data
#csp_storage_options={'account_name':os.environ['CSP_ACCOUNT_NAME'],\
#                 'account_key':os.environ['CSP_BLOB_KEY']}
#csp_fs = fsspec.filesystem('az',\
#                           account_name=csp_storage_options['account_name'],\
#                           account_key=csp_storage_options['account_key'])
#for i in csp_fs.ls('ana-data'):
#    f = pd.read_csv(f'az://{i}', delimiter=';', storage_options=csp_storage_options)
#    f.to_csv(f'az://{i}', storage_options=storage_options, index=False)

# Here we preprocessing of the data for standardized ingestion

In [90]:
for site_no in station_df.site_no:
    station_url = f'az://{container}/{site_no}.csv'
    station_url2 = f'az://{container}/{site_no}_2.csv'
    site_df1_raw = pd.read_csv(station_url, delimiter=',', skiprows=10, storage_options=storage_options)
    translation = pd.read_csv(f'az://{container}/ana_translations.csv', storage_options=storage_options)
    trans = {p:e  for p,e in zip(translation.Portuguese, translation.English)}
    site_df1 = site_df1_raw.rename(columns=trans)
    site_df1 = site_df1.dropna(subset=['Date'])
    site_df1['TimeL'] = site_df1['TimeL'].fillna('01/01/1900 01:00')
    site_df1['Date-Time'] = [d for d in site_df1['Date']]
    site_df1['Date-Time'] = pd.to_datetime(site_df1['Date-Time'],\
                                      format='%d/%m/%Y')

    site_df2_raw = pd.read_csv(station_url2, delimiter=',', skiprows=14, storage_options=storage_options)
    site_df2_raw = site_df2_raw.replace('01/01/1900', '01/01/1900 01:00')
    translation2 = {'Data':'Date','Hora':'Hour','Turbidez':'Turbidity'}
    site_df2 = site_df2_raw.rename(columns=translation2)
    site_df2 = site_df2.dropna(subset=['Date'])
    site_df2['Date-Time-HM'] = [f"{d} {t.split(' ')[1]}" for d,t in zip(site_df2['Date'],site_df2['Hour'])]
    site_df2['Date-Time'] = [d for d in site_df2['Date']]
    site_df2['Date-Time'] = pd.to_datetime(site_df2['Date-Time'],\
                                      format='%d/%m/%Y')
    site_df2 = site_df2[['Date', 'Hour', 'Date-Time','Turbidity']]

    selection = ['Date-Time', 'Discharge', 'Suspended Sediment Concentration (mg/L)', 'Turbidity']
    site_df = site_df1.merge(site_df2, on='Date', how='outer', suffixes=('_',''))
    site_df['Date-Time'] = site_df['Date-Time'].fillna(site_df['Date-Time_'])
    #site_df['Hour'] = site_df['Hour'].fillna(site_df['Hour_'])
    site_df = site_df[selection]
    s = str(site_no).zfill(8)
    write_filename = f'az://{container}/stations/{str(site_no)}.csv'
    print(f'writing to {write_filename}')
    site_df.to_csv(write_filename, index=False, storage_options=storage_options)

writing to az://ana-data/stations/21750000.csv
writing to az://ana-data/stations/22050001.csv
writing to az://ana-data/stations/22500000.csv
writing to az://ana-data/stations/22700000.csv
writing to az://ana-data/stations/22900000.csv
writing to az://ana-data/stations/23100000.csv
writing to az://ana-data/stations/23230000.csv
writing to az://ana-data/stations/23250000.csv
writing to az://ana-data/stations/23700000.csv
writing to az://ana-data/stations/25200000.csv
writing to az://ana-data/stations/26200000.csv
writing to az://ana-data/stations/26300000.csv
writing to az://ana-data/stations/26350000.csv
writing to az://ana-data/stations/26800000.csv
writing to az://ana-data/stations/27500000.csv
writing to az://ana-data/stations/27550000.csv
writing to az://ana-data/stations/28240000.csv
writing to az://ana-data/stations/28300000.csv
writing to az://ana-data/stations/28850000.csv
writing to az://ana-data/stations/29050000.csv
writing to az://ana-data/stations/29100000.csv
writing to az